# Force Calculator

Given a set of parameters, this notebook calculates the required amperage in the wire to flip the magnet.

The following contains a diagram of the setup.

![setup diagram](diagrams/setup.png)

## Imports

In [325]:
from collections import ChainMap
import sympy as sym
import sympy.vector as vec
from scipy import constants

## Parameters

### Parameters

In [469]:
# Solenoid Parameters
solenoid_radius = 0.007 # m
wire_radius = 0.000792 # m
number_of_levels = 1
turns_per_level = 1 # how many coils are at the same level
amperage = 2 # A, max is 5A for wire and 0.012A for esp32, usb-c min is 3A

# Dipole Position Parameters
dipole_z = 0.005 # m
dipole_x = 0.007/2 # m, in the direction of r_hat where x = 0 is the center of the solenoid

# Dipole Properties
dipole_moment_magnitude = 0.0168 # A m^2. Use https://www.kjmagnetics.com/calculator.asp to calculate this.

## Magnetic Field Equation

In [327]:
C = vec.CoordSys3D('C', transformation='cylindrical', vector_names=list('rtk'), variable_names=list('RTK'))

(
D, # Radius of the solenoid
r_0, # Distance along theta = 0 from the center of the solenoid
z_0, # Distance along z from the center of the solenoid
theta, # Angle from the center of the solenoid
pi, # Pi
mu_0, # Permeability of free space
) = sym.symbols('D r_0 z_0 theta pi mu_0', real=True, positive=True)

I = sym.symbols('I', real=True) # Current in the solenoid

# Magnetic field of a solenoid
B_original = mu_0*I*D / (4 * pi) * sym.integrate((z_0 * C.r + (D - r_0) * C.k) / (D**2 - 2*r_0*D*sym.cos(theta) + r_0**2 + z_0**2)**(3/2), (theta, 0, 2*pi))
B_point = B_original.subs({ mu_0: constants.mu_0, pi: constants.pi }).subs({
    r_0: C.R,
    z_0: C.K + z_0,
})

# Print the magnetic field
B_point.simplify()

(1.00000000054438e-7*D*I*(C.K + z_0)*Integral((C.R**2 - 2*C.R*D*cos(theta) + D**2 + (C.K + z_0)**2)**(-1.5), (theta, 0, 6.28318530717959)))*C.r + (1.00000000054438e-7*D*I*(-C.R + D)*Integral((C.R**2 - 2*C.R*D*cos(theta) + D**2 + (C.K + z_0)**2)**(-1.5), (theta, 0, 6.28318530717959)))*C.k

Testing...

In [410]:
# Compare to the answer 6.2831 * 10^-7 k_hat
print(B_point.subs({ C.R: 0, C.K: 0, D: 1, I: 1, z_0: 0 }).simplify())


print(B_point.subs({ C.R: dipole_x, C.K: dipole_z, D: solenoid_radius, I: amperage, z_0: 0 }).simplify())

(6.2831853106e-7)*C.k
(5.61985178789189e-6)*C.r + (1.12397035757838e-5)*C.k


## Force Acting on Dipole

In [478]:
# Magnetic moment of the dipole, as a vector
mu = - dipole_moment_magnitude * C.k
delop = vec.Del()

F_dipole = mu.dot(delop)(B_point)

# Display the dipole force expression
F_dipole.simplify()

KeyboardInterrupt: 

### Use Superposition to Sum the Force

In [471]:
i, j = sym.symbols('i j', cls=sym.Idx)
M, N = sym.symbols('M N', integer=True, positive=True)
F_indexed = F_dipole.subs({ z_0: wire_radius * i, D: solenoid_radius + wire_radius * j })
# Where N is the number of turns per level and M is the number of levels
F_sum = sym.Sum(sym.Sum(F_indexed, (j, 0, N)), (i, 0, M))

Z_vals = [wire_radius * i for i in range(number_of_levels)]
R_vals = [solenoid_radius + wire_radius * j for j in range(turns_per_level)]

F_total = F_sum.subs({
    N: turns_per_level - 1,
    M: number_of_levels - 1,
    I: amperage,
    C.R: dipole_x,
    C.K: dipole_z,
}).doit()

(20*F_total).evalf(4)

0.002097*C.r + 0.005198*C.k

# Torque Acting on a Dipole

In [475]:
T_dipole = mu.cross(B_point)

# Display the dipole force expression
T_dipole.simplify()

(-1.68000000091455e-9*D*I*(C.K + z_0)*Integral((C.R**2 - 2*C.R*D*cos(theta) + D**2 + (C.K + z_0)**2)**(-1.5), (theta, 0, 6.28318530717959)))*C.t

### Use Superposition to Sum the Force

In [479]:
i, j = sym.symbols('i j', cls=sym.Idx)
M, N = sym.symbols('M N', integer=True, positive=True)
T_indexed = T_dipole.subs({ z_0: wire_radius * i, D: solenoid_radius + wire_radius * j })
# Where N is the number of turns per level and M is the number of levels
T_sum = sym.Sum(sym.Sum(T_indexed, (j, 0, N)), (i, 0, M))

Z_vals = [wire_radius * i for i in range(number_of_levels)]
R_vals = [solenoid_radius + wire_radius * j for j in range(turns_per_level)]

T_total = T_sum.subs({
    N: turns_per_level - 1,
    M: number_of_levels - 1,
    I: amperage,
    C.R: dipole_x,
    C.K: dipole_z,
}).doit()

(20*T_total).evalf(4)

(-2.664e-5)*C.t